In [54]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from tensorflow.keras.preprocessing.text import Tokenizer


import pandas as pd
import numpy as np
import regex as re
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer


from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [55]:
#Miscellanous libraries
from collections import Counter

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:

data = pd.read_csv('/content/drive/MyDrive/ start/7004OTT Content Reviews.csv', encoding='UTF-8')
data

,Text,Label
0,"পুরাই অস্থির!!! ""সাকিল, চট্টগ্রাম",1
1,নাজিফা তুষি আপু অবিনয় টা বেস্ট ছিল ওনার জন্য ক...,1
2,পুরাই আগুন অনেক দিনের অপেক্ষায় আছি এই কাজ টা দ...,1
3,"ইন্টারেস্টিং স্টোরি,অভিনয়, সিনেমেটোগ্রাফিও জোস",1
4,এট লিস্ট অনন্ত জলীল এর ১০০ কোটি টাকার মুভির ট্...,1
...,...,...
6998,প্রথমে মিষ্টি এরপর পরকিয়া,2
6999,বৌদি কে লাগাবে,2
7000,এই সিরিজ টা পুরো কোথায় পাওয়া যাবে,0
7001,কাহিনীর গল্প খুব বাজে,2


In [58]:
print("Total Reviews:",len(data),
      "\nTotal Positive Reviews:",len(data[data.Label ==1]),
      "\nTotal Negative Reviews:",len(data[data.Label==2]),
      "\nTotal Neutral Reviews:",len(data[data.Label ==0]))


Total Reviews: 7003 
Total Positive Reviews: 3662 
Total Negative Reviews: 2261 
Total Neutral Reviews: 1080


In [59]:
!pip install bnlp_toolkit
!pip install bangla-stemmer
#importing stemmer
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bangla_stemmer.stemmer import stemmer
import re
btokenizer = BasicTokenizer()

def clean_text(text):
    text = re.sub('[^\u0980-\u09FF]',' ',str(text))
    tokens = btokenizer.tokenize(text)
    stmr = stemmer.BanglaStemmer()
    stm = stmr.stem(tokens)
    filtered = []
    for i in stm:
      
        if i in stopwords:
            continue
    
        if i in punctuations + '‘' + '’':
            continue
       
        filtered.append(i)
    
    return " ".join(filtered)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
data['cleaned'] = data.Text.apply(clean_text)

Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
a

In [61]:
# Apply the function into the dataframe
data['cleaned'] = data['Text'].apply(clean_text)  

# print some cleaned reviews from the dataset
sample_data = [10,100,25,854,966,1500,1589,1700,2000,2500,4000,4500]
for i in sample_data:
     print('Original:\n',data.Text[i],'\nCleaned:\n',
           data.cleaned[i],'\n','Sentiment:-- ',data.Label[i],'\n')

Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..

In [62]:
data

,Text,Label,cleaned
0,"পুরাই অস্থির!!! ""সাকিল, চট্টগ্রাম",1,পুরা অস্থির সাকিল চট্টগ্রাম
1,নাজিফা তুষি আপু অবিনয় টা বেস্ট ছিল ওনার জন্য ক...,1,নাজিফা তুষি আপু অবিনয় টা বেস্ট ওন কান্না আস
2,পুরাই আগুন অনেক দিনের অপেক্ষায় আছি এই কাজ টা দ...,1,পুরা আগুন অপেক্ষায় আছি টা দেখ
3,"ইন্টারেস্টিং স্টোরি,অভিনয়, সিনেমেটোগ্রাফিও জোস",1,ইন্টারেস্টিং স্টোরি অভিনয় সিনেমেটোগ্রাফি জোস
4,এট লিস্ট অনন্ত জলীল এর ১০০ কোটি টাকার মুভির ট্...,1,এট লিস্ট অনন্ত জলীল ১০০ টাকা মুভির ট্রেইল থেক ...
...,...,...,...
6998,প্রথমে মিষ্টি এরপর পরকিয়া,2,প্রথমে মিষ্ এরপর পরকিয়া
6999,বৌদি কে লাগাবে,2,বৌদি লাগাবে
7000,এই সিরিজ টা পুরো কোথায় পাওয়া যাবে,0,সিরিজ টা পুরো কোথায় পাওয়া
7001,কাহিনীর গল্প খুব বাজে,2,কাহিনীর গল্প বাজে


In [63]:
data['length'] = data['cleaned'].apply(lambda x:len(x.split()))

# Remove the reviews with least words
dataset = data.loc[data.length>2]
dataset = data.reset_index(drop = True)

In [64]:
print("After Cleaning:","\nRemoved {} Small Reviews".format(len(data)-len(dataset)),
      "\nTotal Reviews:",len(dataset))

After Cleaning: 
Removed 0 Small Reviews 
Total Reviews: 7003


In [65]:
print("Maximum Length of a Review:",max(dataset.length))
print("Minimum Length of a Review:", min(dataset.length))
print("Average Length of a Review:",round(np.mean(dataset.length),0))

Maximum Length of a Review: 156
Minimum Length of a Review: 0
Average Length of a Review: 8.0


In [66]:
dataset=dataset[['cleaned','Label']]
dataset.head(5)

,cleaned,Label
0,পুরা অস্থির সাকিল চট্টগ্রাম,1
1,নাজিফা তুষি আপু অবিনয় টা বেস্ট ওন কান্না আস,1
2,পুরা আগুন অপেক্ষায় আছি টা দেখ,1
3,ইন্টারেস্টিং স্টোরি অভিনয় সিনেমেটোগ্রাফি জোস,1
4,এট লিস্ট অনন্ত জলীল ১০০ টাকা মুভির ট্রেইল থেক ...,1


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(dataset['cleaned']).toarray()

In [68]:
y = dataset['Label'].values

In [69]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 1)

In [70]:
x_train.shape

(5252, 1733)

In [71]:
y_train.shape

(5252,)

In [72]:
o = 0
for i in y_test:
    if i == 1:
        o = o + 1
print(o)

881


In [73]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [74]:
ann = Sequential()

In [75]:
#Add the input layer and the first hidden layer
ann.add(Dense(units = 64,activation='relu',input_dim = 1733))
#Add the second hidden layer
ann.add(Dense(units = 64,activation='relu'))
#Add the output layer
ann.add(Dense(units = 1,activation='softmax'))

In [76]:
#Compile the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                110976    
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 115,201
Trainable params: 115,201
Non-trainable params: 0
_________________________________________________________________


In [77]:
ann.fit(x_train,y_train,validation_split=.3,batch_size=100,epochs=30)

Epoch 1/30
37/37 [==============================] - 1s 12ms/step - loss: 0.4238 - accuracy: 0.5291 - val_loss: 0.0246 - val_accuracy: 0.5305
Epoch 2/30
37/37 [==============================] - 0s 7ms/step - loss: -0.5862 - accuracy: 0.5291 - val_loss: -1.0770 - val_accuracy: 0.5305
Epoch 3/30
37/37 [==============================] - 0s 8ms/step - loss: -2.4911 - accuracy: 0.5291 - val_loss: -3.5281 - val_accuracy: 0.5305
Epoch 4/30
37/37 [==============================] - 0s 7ms/step - loss: -7.2934 - accuracy: 0.5291 - val_loss: -9.2618 - val_accuracy: 0.5305
Epoch 5/30
37/37 [==============================] - 0s 7ms/step - loss: -17.1554 - accuracy: 0.5291 - val_loss: -19.9231 - val_accuracy: 0.5305
Epoch 6/30
37/37 [==============================] - 0s 7ms/step - loss: -34.1030 - accuracy: 0.5291 - val_loss: -37.1788 - val_accuracy: 0.5305
Epoch 7/30
37/37 [==============================] - 0s 7ms/step - loss: -60.4053 - accuracy: 0.5291 - val_loss: -62.1696 - val_accuracy: 0.5305
E

In [78]:
yp = ann.predict(x_test)
yp[:5]

55/55 [==============================] - 0s 2ms/step


array([[0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994]], dtype=float32)

In [79]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [80]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       270
           1       0.50      1.00      0.67       881
           2       0.00      0.00      0.00       600

    accuracy                           0.50      1751
   macro avg       0.17      0.33      0.22      1751
weighted avg       0.25      0.50      0.34      1751



In [81]:
from keras.layers import LSTM , Embedding
import keras
from keras.models import Sequential
from keras.layers import Dense

In [82]:
from tensorflow.keras.layers import SimpleRNN

# define the model
model = Sequential()
model.add(Embedding(32000, 24, input_length=1733))#24--86
model.add(SimpleRNN(24, return_sequences=False)) #24--86
model.add(Dense(1, activation='sigmoid')) #2--86%

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #optimizer='rmsprop' --86

# summarize the model
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1733, 24)          768000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 24)                1176      
                                                                 
 dense_15 (Dense)            (None, 1)                 25        
                                                                 
Total params: 769,201
Trainable params: 769,201
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
# For conv1D dimentionality should be 187X1 where 187 is number of features and 1 = 1D Dimentionality of data
x_train = x_train.reshape(len(x_train),x_train.shape[1],1)
x_test = x_test.reshape(len(x_test),x_test.shape[1],1)

In [84]:
x_train.shape

(5252, 1733, 1)

In [85]:
from keras.models import Sequential
from keras.layers import Dense # for fully connected layers dense will be used
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam

# avoid overfitting by normalizing the samples
from tensorflow.keras.layers import BatchNormalization
#from keras.layers.normalization import BatchNormalization

In [86]:
def build_model():
    model = Sequential()
    
    # Filters = Units in Dense Total number of Neurons
    # Padding = 'same' , zero-padding, Add zero pixels all around input data
    model.add(Conv1D(filters = 64, kernel_size = 6, activation='relu', padding = 'same', input_shape = (1716, 1))) #we pass individual values hence not 100000,187,1
    
    # Normalization to avoid overfitting
    model.add(BatchNormalization())
    
    # Pooling 
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    model.add(Conv1D(filters = 64, kernel_size = 6, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    model.add(Conv1D( filters = 64, kernel_size = 6, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    # Flatten 
    model.add(Flatten())

    # Fully connected layer
    # input layer
    model.add(Dense(units = 20, activation='relu'))
    
    # Hidden Layer
    model.add(Dense(units = 20, activation='relu'))
    
    # Output Layer
    model.add(Dense(units = 1, activation='softmax'))

    # loss = 'categorical_crossentropy'
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [87]:
model = build_model()

In [88]:
# This is for one sample, i.e. one row
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 1716, 64)          448       
                                                                 
 batch_normalization_6 (Batc  (None, 1716, 64)         256       
 hNormalization)                                                 
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 858, 64)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 858, 64)           24640     
                                                                 
 batch_normalization_7 (Batc  (None, 858, 64)          256       
 hNormalization)                                                 
                                                      

In [89]:
history = model.fit(x_train, y_train, epochs = 30, batch_size = 20, validation_data=(x_test, y_test))

Epoch 1/30


ValueError: ignored

In [ ]:
# evaluate ECG Test Data
model.evaluate(x_test, y_test)

In [ ]:
#Importing the libraries  
import numpy as nm  
import matplotlib.pyplot as mtp  
import pandas as pd 

In [ ]:
# Splitting the dataset into the Training set and Test set  
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 1)  


In [ ]:
 #Feature Scaling  
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)